In [9]:

lr= 0.000025341814210920752
epoch = 68
resize = 148
dropout= 0.02071552291233333


maxpooling= [2,5]

batch_size = 128
num_filters= [18,5]
kernel_size = [10,8]
resize = 148

  
   

In [1]:
import torch
import torch.nn as nn

In [3]:
class BMIModel(nn.Module):
    def __init__(self, resize = 148, num_filters = [18,5], kernel_size =[10,8], maxpooling = [2,5]):
        super().__init__()
        self.resize = resize
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.maxpooling = maxpooling
        
        
        # input -> conv2d -> batchnorm -> maxpool2d ->conv2d -> batchnorm -> maxpool 
        
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)

        self.conv1 = nn.Conv2d(
                            in_channels= 3, 
                            out_channels=self.num_filters[0], 
                            kernel_size=self.kernel_size[0],
                            )

        self.conv2 = nn.Conv2d(
                            in_channels=self.num_filters[0], 
                            out_channels=self.num_filters[1], 
                            kernel_size=self.kernel_size[1],
                            )
        # self.conv3 = nn.Conv2d(
        #                     in_channels=self.num_filters[1], 
        #                     out_channels=1, 
        #                     kernel_size=3,
        #                     )
        # nn.init.xavier_uniform_(self.conv1.weight)
        # nn.init.zeros_(self.conv1.bias)
        # nn.init.xavier_uniform_(self.conv2.weight)
        # nn.init.zeros_(self.conv2.bias)

        self.f_activation1 = nn.ReLU()
        self.f_activation2 = nn.ReLU()
        
        


        # torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)
        self.batchNorm1 = nn.BatchNorm2d(num_features=self.num_filters[0])
        self.batchNorm2 = nn.BatchNorm2d(num_features=self.num_filters[1])

        # torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        
        self.maxpooling1 = nn.MaxPool2d(kernel_size=self.maxpooling[0])
        self.maxpooling2 = nn.MaxPool2d(kernel_size=self.maxpooling[1])
        
        self.h_conv1 = int((self.resize- self.kernel_size[0]) + 1)
        #self.w_conv1 = int((self.shape_data[2] - self.kernel_size[0]) + 1)

        self.h_maxpool1 = int((self.h_conv1 - self.maxpooling[0])/self.maxpooling[0] + 1)
        #self.w_maxpool1 = int((self.w_conv1 - self.maxpooling[0])/self.maxpooling[0] + 1)  

        self.h_conv2 = int((self.h_maxpool1 - self.kernel_size[1]) + 1)
        #self.w_conv2 = int((self.w_maxpool1 - self.kernel_size[1]) + 1)

        self.h_maxpool2= int((self.h_conv2 - self.maxpooling[1])/self.maxpooling[1] + 1)
        #self.w_maxpool2 = int((self.w_conv2 - self.maxpooling[1])/self.maxpooling[1] + 1)  

        # self.h_conv3 = int((self.h_maxpool2 - 3) + 1)
        # self.w_conv3 = int((self.w_maxpool2 - 3) + 1)
        
        
        
        self.linear1 = nn.Linear(self.h_maxpool2*self.h_maxpool2*num_filters[1], 120)
        self.f_activation3 = nn.ReLU()
        self.linear2 = nn.Linear(120, 84)
        self.f_activation4= nn.ReLU()
        self.linear3 = nn.Linear(84, 1)
        
        
        
        
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.f_activation1(x)
        x = self.batchNorm1(x)
        x = self.maxpooling1(x)
        
        
        x = self.conv2(x)
        x = self.f_activation2(x)
        x = self.batchNorm2(x)
        x = self.maxpooling2(x)
        x = torch.flatten(x, start_dim=1)
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        
        return x

In [4]:
loaded_model = BMIModel()
loaded_model.load_state_dict(torch.load('/home/quanhhh/Documents/pre_bmi/result/(10,8)kernel_(18,5)filter_(2,5)maxpool_2.5341814210920752e-05lr.pt')) # it takes the loaded dictionary, not the path file itself
loaded_model.eval()

#print(loaded_model.state_dict())

OrderedDict([('conv1.weight', tensor([[[[-0.0021,  0.0003, -0.0195,  ...,  0.0546,  0.0135, -0.0201],
          [ 0.0082, -0.0041,  0.0256,  ..., -0.0020,  0.0127, -0.0010],
          [-0.0031, -0.0488, -0.0479,  ..., -0.0523, -0.0550,  0.0245],
          ...,
          [ 0.0573, -0.0262, -0.0361,  ...,  0.0161, -0.0194, -0.0317],
          [-0.0342, -0.0181,  0.0294,  ...,  0.0574, -0.0546, -0.0222],
          [-0.0329,  0.0583,  0.0333,  ...,  0.0196, -0.0285, -0.0349]],

         [[-0.0006,  0.0343,  0.0495,  ...,  0.0229, -0.0216, -0.0105],
          [ 0.0276,  0.0244,  0.0385,  ..., -0.0291, -0.0196,  0.0363],
          [ 0.0501,  0.0207, -0.0434,  ...,  0.0115, -0.0538, -0.0030],
          ...,
          [ 0.0012,  0.0399,  0.0093,  ..., -0.0413,  0.0300,  0.0134],
          [ 0.0173, -0.0122,  0.0539,  ...,  0.0365,  0.0524,  0.0512],
          [ 0.0292,  0.0089, -0.0405,  ...,  0.0471, -0.0507, -0.0413]],

         [[-0.0598,  0.0286,  0.0096,  ..., -0.0417, -0.0413,  0.0350],


In [17]:
from torchvision import datasets, transforms
from PIL import Image
transform = transforms.Compose([
    transforms.Resize((resize, resize)),  # Chọn kích thước ảnh sau khi augment
    transforms.ToTensor(),  # Chuyển ảnh sang tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Chuẩn hóa tensor
])

In [19]:
with torch.no_grad():
   
    image = Image.open('thanh1.jpg').convert("RGB")
    image = transform(image)
    
    image = image.unsqueeze_(0)
    #y =image.size()
    y = loaded_model(image)

    print(y)

tensor([[24.6832]])
